In [1]:
import spacy

nlp = spacy.load("en_core_sci_sm")
doc = nlp("Alterations in the hypocretin receptor 2 and preprohypocretin genes produce narcolepsy in some animals.")
for token in doc:
    print(token.text, token.lemma_, token.pos_)

Alterations alteration NOUN
in in ADP
the the DET
hypocretin hypocretin NOUN
receptor receptor NOUN
2 2 NUM
and and CCONJ
preprohypocretin preprohypocretin NOUN
genes gene NOUN
produce produce VERB
narcolepsy narcolepsy NOUN
in in ADP
some some DET
animals animal NOUN
. . PUNCT


/Users/morris/projects/KG-GNN-pubmed/.venv/lib/python3.11/site-packages/spacy/language.py:2195: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]
